In [4]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from meteostat import Point, Hourly
from datetime import datetime
from pandas.api.types import CategoricalDtype

#set notebook options
pd.options.mode.chained_assignment = None 
pd.set_option('display.max_columns', None)
plt.rcParams.update({'font.size': 10})
sns.set_style("white")
plt_color = 'cadetblue'

In [13]:
# import bokeh for interactive plots
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, CategoricalColorMapper
from bokeh.palettes import Spectral6
from bokeh.transform import factor_cmap
from bokeh.layouts import gridplot

# import math
import math

# import bokeh panel and tabs

from bokeh.models import TabPanel, Tabs


output_notebook()


Loading BokehJS ...

In [2]:
df = pd.read_csv('./Data/Motor_Vehicle_Collisions_Crashes.csv')

In [3]:
df['HOUR'] = pd.to_datetime(df['CRASH TIME'], format='%H:%M').dt.hour
df['MINUTE'] = pd.to_datetime(df['CRASH TIME'], format='%H:%M').dt.minute
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])
df['YEAR'] = df['CRASH DATE'].dt.year
df['MONTH'] = df['CRASH DATE'].dt.month
df['DAY'] = df['CRASH DATE'].dt.day
df['WEEKDAY'] = df['CRASH DATE'].dt.day_name()
dayorder = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['WEEKDAY'] = df['WEEKDAY'].astype(CategoricalDtype(categories=dayorder, ordered=True))
df['HOUR OF WEEK'] = df['CRASH DATE'].dt.dayofweek * 24 + (df.HOUR + 1).astype(int)

# drop data incomplete years
df = df[(df.YEAR > 2020) & (df.YEAR < 2024)]
df = df[(df.LONGITUDE != 0.000000) | (df.LATITUDE != 0.000000)]

#sort data on CRASH DATE
df = df.sort_values(by=['CRASH DATE']).reset_index(drop=True)

# keep data from 2017-2022
#df = df[(df['YEAR'] >= 2017) & (df['YEAR'] <= 2022)]

In [12]:
print(df.shape)

(235763, 39)


In [6]:
# count number of collisions with no injuries or fatalities
df['NO INJURIES'] = np.where(df['NUMBER OF PERSONS INJURED'] == 0, 1, 0)
df['NO FATALITIES'] = np.where(df['NUMBER OF PERSONS KILLED'] == 0, 1, 0)
df['NO INJURIES OR FATALITIES'] = np.where((df['NUMBER OF PERSONS INJURED'] == 0) & (df['NUMBER OF PERSONS KILLED'] == 0), 1, 0)

df.sum()

CRASH TIME                       4:356:4519:423:3022:406:503:006:211:402:4010:4...
LATITUDE                                                            8808503.703636
LONGITUDE                                                         -15987794.582183
NUMBER OF PERSONS INJURED                                                 114982.0
NUMBER OF PERSONS KILLED                                                     629.0
NUMBER OF PEDESTRIANS INJURED                                                18613
NUMBER OF PEDESTRIANS KILLED                                                   281
NUMBER OF CYCLIST INJURED                                                    10801
NUMBER OF CYCLIST KILLED                                                        48
NUMBER OF MOTORIST INJURED                                                   80644
NUMBER OF MOTORIST KILLED                                                      266
COLLISION_ID                                                         1061070488073
HOUR

In [7]:
df_orig = df.copy()

In [9]:
df = df_orig
df.shape

(235763, 39)

I want a plot where the focus is on the factor. It should be grouped by severity (material damage, injured and killed), type of transportation (motorist, cyclist, pedestrian) and borough.

In [26]:
# create columns that indicate if only material damage, injuries or fatalities occured
df['MATERIAL DAMAGE'] = np.where((df['NUMBER OF PERSONS INJURED'] == 0) & (df['NUMBER OF PERSONS KILLED'] == 0), 1, 0)
df['INJURIES'] = np.where((df['NUMBER OF PERSONS INJURED'] > 0) & (df['NUMBER OF PERSONS KILLED'] == 0), 1, 0)
df['FATALITIES'] = np.where((df['NUMBER OF PERSONS INJURED'] >= 0) & (df['NUMBER OF PERSONS KILLED'] > 0), 1, 0)


# the sum of MATERIAL DAMAGE, INJURIES and FATALITIES should be equal to df.shape = 235763
df['MATERIAL DAMAGE'].sum() + df['INJURIES'].sum() + df['FATALITIES'].sum()


235762

In [ ]:
# categorize collisions two fold: by type and severity

In [30]:
# categorize data depending on whether a motorist, cyclist, or pedestrian was involved
df["PEDESTRIAN_INJURED"] = np.where(df["NUMBER OF PEDESTRIANS INJURED"] > 0, 1, 0)
df["PEDESTRIAN_KILLED"] = np.where(df["NUMBER OF PEDESTRIANS KILLED"] > 0, 1, 0)
df["CYCLIST_INJURED"] = np.where(((df["NUMBER OF CYCLIST INJURED"] > 0) & (df["PEDESTRIAN_INJURED"] <= 0)), 1, 0)
df["CYCLIST_KILLED"] = np.where(((df["NUMBER OF CYCLIST KILLED"] > 0) & (df["PEDESTRIAN_KILLED"] <= 0)), 1, 0)
df["MOTORIST_INJURED"] = np.where((df["NUMBER OF MOTORIST INJURED"] > 0) & (df["CYCLIST_INJURED"] == 0) & (df["PEDESTRIAN_INJURED"] == 0), 1, 0)
df["MOTORIST_KILLED"] = np.where((df["NUMBER OF MOTORIST KILLED"] > 0) & (df["CYCLIST_KILLED"] == 0) & (df["PEDESTRIAN_KILLED"] == 0), 1, 0)


df.sum()


CRASH TIME                       4:356:4519:423:3022:406:503:006:211:402:4010:4...
LATITUDE                                                            8808503.703636
LONGITUDE                                                         -15987794.582183
NUMBER OF PERSONS INJURED                                                 114982.0
NUMBER OF PERSONS KILLED                                                     629.0
NUMBER OF PEDESTRIANS INJURED                                                18613
NUMBER OF PEDESTRIANS KILLED                                                   281
NUMBER OF CYCLIST INJURED                                                    10801
NUMBER OF CYCLIST KILLED                                                        48
NUMBER OF MOTORIST INJURED                                                   80644
NUMBER OF MOTORIST KILLED                                                      266
COLLISION_ID                                                         1061070488073
HOUR

In [31]:
df["PEDESTRIAN_KILLED"].sum()+df["PEDESTRIAN_INJURED"].sum()+df["CYCLIST_KILLED"].sum()+df["CYCLIST_INJURED"].sum()+df["MOTORIST_KILLED"].sum()+df["MOTORIST_INJURED"].sum()

82146

In [32]:
# make three dataframes for material damage, injuries and fatalities
df_material_damage = df[['CONTRIBUTING FACTOR VEHICLE 1', 'MATERIAL DAMAGE']].groupby('CONTRIBUTING FACTOR VEHICLE 1').sum().reset_index()
df_injured = df[['CONTRIBUTING FACTOR VEHICLE 1', 'MOTORIST_INJURED', 'CYCLIST_INJURED', 'PEDESTRIAN_INJURED']].groupby('CONTRIBUTING FACTOR VEHICLE 1').sum().reset_index()
df_killed = df[['CONTRIBUTING FACTOR VEHICLE 1', 'MOTORIST_KILLED', 'CYCLIST_KILLED', 'PEDESTRIAN_KILLED']].groupby('CONTRIBUTING FACTOR VEHICLE 1').sum().reset_index()


In [64]:
# sort the df by the sum of the three categories
df_injured['sum'] = df_injured['MOTORIST_INJURED'] + df_injured['CYCLIST_INJURED'] + df_injured['PEDESTRIAN_INJURED']
df_killed['sum'] = df_killed['MOTORIST_KILLED'] + df_killed['CYCLIST_KILLED'] + df_killed['PEDESTRIAN_KILLED']
df_injured = df_injured.sort_values(by=['sum'], ascending=True)
df_killed = df_killed.sort_values(by=['sum'], ascending=True)
df_material_damage = df_material_damage.sort_values(by=['MATERIAL DAMAGE'], ascending=True)

# drop the rows with 'Unspecified' as the contributing factor
df_injured = df_injured[df_injured['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']
df_killed = df_killed[df_killed['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']
df_material_damage = df_material_damage[df_material_damage['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']

# only keep the top 8 contributing factors
df_injured = df_injured.head(8)
df_killed = df_killed.head(8)
df_material_damage = df_material_damage.head(8)


# create a ColumnDataSource object for each category
source_material_damage = ColumnDataSource(df_material_damage)
source_injured = ColumnDataSource(df_injured)
source_killed = ColumnDataSource(df_killed)

# create a figure object for each category
p_material_damage = figure(y_range=df_material_damage['CONTRIBUTING FACTOR VEHICLE 1'], height=500, width=1000, title='Most Frequent Contributing Factors for Material Damage', toolbar_location=None, tools='')
p_injured = figure(y_range=df_injured['CONTRIBUTING FACTOR VEHICLE 1'], height=500, width=1000, title='Most Frequent Contributing Factors for Injured', toolbar_location=None, tools='')
p_killed = figure(y_range=df_killed['CONTRIBUTING FACTOR VEHICLE 1'], height=500, width=1000, title='Most Frequent Contributing Factors for Killed', toolbar_location=None, tools='')
#p_material_damage.xaxis.major_label_orientation = math.pi/2
#p_injured.xaxis.major_label_orientation = math.pi/2
#p_killed.xaxis.major_label_orientation = math.pi/2

# remove x < 0 axis
p_material_damage.x_range.start = 0
p_injured.x_range.start = 0
p_killed.x_range.start = 0

# specify pleasent colors for each category
colors = ["#4271AE", "#F5A623", "#4CAF50"]



# create a stacked bar plot for each category
p_material_damage.hbar(y='CONTRIBUTING FACTOR VEHICLE 1', left=0, right='MATERIAL DAMAGE', height=0.8, color=colors[0], source=source_material_damage)
p_injured.hbar_stack(['MOTORIST_INJURED', 'CYCLIST_INJURED', 'PEDESTRIAN_INJURED'], y='CONTRIBUTING FACTOR VEHICLE 1', height=0.8, color=colors, source=source_injured, legend_label=['Motorist', 'Cyclist', 'Pedestrian'])
p_killed.hbar_stack(['MOTORIST_KILLED', 'CYCLIST_KILLED', 'PEDESTRIAN_KILLED'], y='CONTRIBUTING FACTOR VEHICLE 1', height=0.8 , color=colors, source=source_killed, legend_label=['Motorist', 'Cyclist', 'Pedestrian'])

# add hover tool to each figure
hover_material_damage = HoverTool(tooltips=[('Contributing Factor', '@{CONTRIBUTING FACTOR VEHICLE 1}'), ('Material Damage', '@{MATERIAL DAMAGE}')])
hover_injured = HoverTool(tooltips=[('Contributing Factor', '@{CONTRIBUTING FACTOR VEHICLE 1}'), ('Motorist', '@{MOTORIST_INJURED}'), ('Cyclist', '@{CYCLIST_INJURED}'), ('Pedestrian', '@{PEDESTRIAN_INJURED}')])
hover_killed = HoverTool(tooltips=[('Contributing Factor', '@{CONTRIBUTING FACTOR VEHICLE 1}'), ('Motorist', '@{MOTORIST_KILLED}'), ('Cyclist', '@{CYCLIST_KILLED}'), ('Pedestrian', '@{PEDESTRIAN_KILLED}')])

p_material_damage.add_tools(hover_material_damage)
p_injured.add_tools(hover_injured)
p_killed.add_tools(hover_killed)

# add legend to each figure
p_injured.legend.location = 'bottom_right'
p_injured.legend.orientation = 'horizontal'
p_killed.legend.location = 'bottom_right'
p_killed.legend.orientation = 'horizontal'

# create tabs for each category
tab_material_damage = TabPanel(child=p_material_damage, title='Material Damage')
tab_injured = TabPanel(child=p_injured, title='Injured')
tab_killed = TabPanel(child=p_killed, title='Killed')

# create a Tabs object with the two tabs
tabs = Tabs(tabs=[tab_material_damage, tab_injured, tab_killed])

# show the plot
show(tabs)
